In [7]:
import cv2
import numpy as np
import os
import glob

# Function to apply mask to an image
def apply_mask(original_image, mask_image):
    # Convert the mask to a binary mask if it's not already
    _, binary_mask = cv2.threshold(mask_image, 128, 255, cv2.THRESH_BINARY)

    # Apply the mask to keep the person and remove the background
    foreground = cv2.bitwise_and(original_image, original_image, mask=binary_mask)
    return foreground

# Function to process images in a directory using corresponding masks
def process_images_with_masks(images_dir, masks_dir, output_dir):
    # Find all original images
    original_img_paths = glob.glob(os.path.join(images_dir, '*.png'))
    
    processed_images = []
    for original_path in original_img_paths:
        # Corresponding mask path
        filename = os.path.basename(original_path)
        mask_path = os.path.join(masks_dir, filename.replace('.png', '.png'))
        
        # Load images
        original_image = cv2.imread(original_path)
        mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Apply mask
        result_image = apply_mask(original_image, mask_image)

        # Save the result
        output_path = os.path.join(output_dir, f"processed_{filename}")
        cv2.imwrite(output_path, result_image)
        processed_images.append(output_path)

    return processed_images

# Function to create a video from images
def create_video_from_images(image_paths, video_path, frame_rate=30):
    first_image = cv2.imread(image_paths[0])
    height, width, layers = first_image.shape
    video_size = (width, height)

    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, video_size)

    for image in image_paths:
        video.write(cv2.imread(image))

    video.release()

# Example usage:
images_dir = './input/frames'
masks_dir = './input/masks'
output_dir = './output/processed_images'
video_path = './output/removed _background.mp4'
removed_images = process_images_with_masks(images_dir, masks_dir, output_dir)
create_video_from_images(removed_images, video_path)
